# Newton Methods in Scipy

## Learning Objectives

After studying this notebook, completing the activities, and asking questions in class, you should be able to:
* Use numpy to solve the flash example problem from the [previous notebook](../04-publish/05-System-of-Equations-Newton-Method.ipynb).

In [1]:
# load libraries
import numpy as np
from scipy import optimize

## Using ``scipy`` instead

`numpy` and `scipy` offer a few different implementations of Newton's method. However, we found these to be unreliable in the past. Instead, we recommend either using the Newton solver we put together in the [last notebook](../04-publish/05-System-of-Equations-Newton-Method.ipynb) or Pyomo (future notebook).

However, we will show how this is done using `scipy.optimize.newton` in this notebook.

In [ ]:
# documentation for Newton's method in scipy
help(optimize.newton)

## Polynomial Example Revisited Using Scipy

$$ c(x) = 3 x^3 + 2 x^2 - 5 x - 20 $$

![Newton Polynomial](../../media/newton_polynomial.png)
    
In the following cells we will demonstrate the use of Scipy to perform the Newton-Raphson method for the polynomial example from [this previous notebook](../04-publish/02-Newton-Raphson-Method-in-One-Dimension.ipynb).

First, we need to run the code we already developed for the polynomial function and its derivative.

In [6]:
def nonlinear_function(x):
    ''' compute a nonlinear function for demonstration
    Arguments:
        x: scalar
    Returns:
        c(x): scalar
    '''
    return 3*x**3 + 2*x**2 - 5*x-20

def Dnonlinear_function(x):
    ''' compute 1st derivative of nonlinear function for demonstration
    Arguments:
        x: scalar
    Returns:
        c'(x): scalar
    '''
    return 9*x**2 + 4*x - 5

Next, we need to provide an initial guess as we did before we the code we deveoped for Newton's Method.

In [4]:
guess = 1

Now we can run Newton's Method through `scipy.optimize.newton`.

In [5]:
polysln = optimize.newton(func=nonlinear_function, x0 = guess, fprime=Dnonlinear_function)
print("The root is found at",polysln)

The root is found at 1.9473052357731322


## Flash Problem Revisted Using Scipy

Recall the flash problem from [this previous notebook](../04-publish/01-Modeling-Systems-of-Nonlinear-Equations.ipynb) that we solved in the [last notebook](../04-publish/05-Newton-Raphson-Methods-for-Systems-of-Equations.ipynb) using Newton's Method.  In the following cells we will demonstrate the use of Scipy to perform the Newton-Raphson method.

![flash](../../media/flash-system.png)

**Parameters (given)**:
* $F$ feed inlet flowrate, mol/time or kg/time
* $z_1$ composition of species 1 in feed, mol% or mass%
* $z_2$ composition of species 2 in feed, mol% or mass%
* $K_1$ partion coefficient for species 1, mol%/mol% or mass% / mass%
* $K_2$ partion coefficient for species 2, mol%/mol% or mass% / mass%

**Variables (unknown)**:
* $L$ liquid outlet flowrate, mol/time or kg/time
* $x_1$ composition of species 1 in liquid, mol% or mass%
* $x_2$ composition of species 2 in liquid, mol% or mass%
* $V$ vapor outlet flowrate, mol/time or kg/time
* $y_1$ composition of species 1 in vapor, mol% or mass%
* $y_2$ composition of species 2 in vapor, mol% or mass%

How to solve the flash problem and other multidimensional problem with $n$ unknown variables and $n$ nonlinear equations?

### System of Equations in Canonical Form

$$\mathbf{F}(\mathbf{x}) = \begin{pmatrix}
L + V - F\\
Vy_1 + L x_1 - F z_1 \\
V y_2 + L x_2 - F z_2 \\
y_1 - K_1 x_1 \\
y_2 - K_2 x_2 \\
y_1 + y_2 - x_1 - x_2
\end{pmatrix},$$

with $\mathbf{x} = (L, V, x_1, x_2, y_1, y_2).$

First run the code below to define the system of nonlinear equations given in the [previous notebook](../04-publish/05-Newton-Raphson-Methods-for-Systems-of-Equations.ipynb).

In [7]:
# nonlinear canonical system of equations
def my_f(x):
    ''' Nonlinear system of equations in conancial form F(x) = 0
    Copied from previous notebook.
    
    Arg:
        x: vector of variables
        
    Returns:
        r: residual, F(x)
    
    '''

    # Initialize residuals
    r = np.zeros(6)
    
    # given data
    F = 1.0
    z1 = 0.5
    z2 = 0.5
    K1 = 3
    K2 = 0.05
    
    # copy values from x to more meaningful names
    L = x[0]
    V = x[1]
    x1 = x[2]
    x2 = x[3]
    y1 = x[4]
    y2 = x[5]
    
    # equation 1: overall mass balance
    r[0] = V + L - F
    
    # equations 2 and 3: component mass balances
    r[1] = V*y1 + L*x1 - F*z1
    r[2] = V*y2 + L*x2 - F*z2
    
    # equation 4 and 5: equilibrium
    r[3] = y1 - K1*x1
    r[4] = y2 - K2*x2
    
    # equation 6: summation
    r[5] = (y1 + y2) - (x1 + x2)
    # This is known as the Rachford-Rice formulation for the summation constraint
    
    return r

Next, we need to define the initial guess as we did with our Newton System in the [previous notebook](../04-publish/05-Newton-Raphson-Methods-for-Systems-of-Equations.ipynb).

In [8]:
# initial guess
x0 = np.array([0.5, 0.5, 0.55, 0.45, 0.65, 0.35])

Now we can run Newton's Method through `scipy.optimize.newton`.  Note that the secant method is used if a derivative is not given to the `scipy.optimize.newton` function, as is the case below.

In [9]:
xsln1 = optimize.newton(func=my_f, x0=x0)
print("Solution using scipy:\n",xsln1)

Solution using scipy:
 [ 5.00004967e-01  5.36467908e-01  1.09276056e+10  3.55480055e+00
 -5.38704523e+36  3.50020142e-01]


C:\Users\ebrea\anaconda3\envs\jbook\lib\site-packages\scipy\optimize\_zeros_py.py:466: RuntimeWarning: some failed to converge after 50 iterations
  warnings.warn(msg, RuntimeWarning)


The correct answer should be <code> [0.72368421 0.27631579 0.3220339  0.6779661  0.96610169 0.03389831] </code>

However, we see that all of the vvalues are incorrect and the third and fifth values diverged.  This illustrates why we prefer to use our own code for Newton's method over scipy.

Now let's try giving it a second guess of x1 that is close to what we know to be the correct answer.

In [11]:
x1 = np.array([0.72, 0.27, 0.32, 0.67, 0.96, 0.03])
xsln2 = optimize.newton(func=my_f, x0=x0, x1=x1)
print("Solution using scipy:\n",xsln2)

Solution using scipy:
 [ 5.00004967e-01  5.36467908e-01  1.09276056e+10  3.55480055e+00
 -5.38704523e+36  3.50020142e-01]


Even adding a second guess closer to the correct root doesn't seem to help.

Let's try one more thing.  Now, we're going to call a finite difference function for the derivative so that we aren't defaulting to the secant method with scipy.

In [24]:
central = lambda x,i:(my_f(x+i) - my_f(x-i))/2/i
i = np.linspace(0,1,1000)
xsln3 = optimize.newton(func=my_f, fprime=central, x0=x0, x1=x1, args=(i))
print("Solution using scipy:\n",xsln3)

TypeError: my_f() takes 1 positional argument but 1001 were given